In [1]:
import  pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib


import json
import warnings
warnings.filterwarnings('ignore')

In [2]:
X_train = joblib.load("../data/models/X_train_pre.joblib")
X_val   = joblib.load("../data/models/X_val_pre.joblib")
X_test  = joblib.load("../data/models/X_test_pre.joblib")


In [3]:

y_train = pd.read_parquet("../data/Gold/y_train.parquet").iloc[:,0].to_numpy()
y_val   = pd.read_parquet("../data/Gold/y_val.parquet").iloc[:,0].to_numpy()
y_test  = pd.read_parquet("../data/Gold/y_test.parquet").iloc[:,0].to_numpy()


In [4]:
preprocessor  = joblib.load("../data/models/preprocessor.joblib")
NUMERIC       = joblib.load("../data/models/numeric_cols.joblib")
CATEGORICAL   = joblib.load("../data/models/categorical_cols.joblib")
feat_names    = joblib.load("../data/models/prepared_feature_names.joblib")
# class_weights = joblib.load("../data/models/class_weights.joblib")

In [5]:
if hasattr(X_train, "tocsr"):
    print("Converting sparse matrix to dense array for modeling...")
else:
    print("Data is already in dense array format.")

Data is already in dense array format.


In [6]:
print(" Shapes:")
print(f"  Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

print("\n Positive Samples:")
print(f"  Train: {int(y_train.sum())}, Val: {int(y_val.sum())}, Test: {int(y_test.sum())}")

print("\nClass Ratios:")
print(f"  Train: {y_train.mean():.4f}, Val: {y_val.mean():.4f}, Test: {y_test.mean():.4f}")


 Shapes:
  Train: (19139453, 49), Val: (6379894, 49), Test: (6378891, 49)

 Positive Samples:
  Train: 15536, Val: 9054, Test: 10640

Class Ratios:
  Train: 0.0008, Val: 0.0014, Test: 0.0017


### Experimental Comparison: Class Weighting vs Resampling

Class weights

In [7]:
# positives negatives and class weights
pos = int(y_train.sum())
neg = int((y_train == 0).sum())
total = pos + neg

class_weights = {
    0: neg / total,
    1: pos / total
}

scale_pos_weight = neg / pos if pos > 0 else 1.0

print("Class Weights:", class_weights)
print("Scale Pos Weight:", round(scale_pos_weight, 2))


Class Weights: {0: 0.9991882735624681, 1: 0.00081172643753194}
Scale Pos Weight: 1230.94


Evaluation block

In [8]:
from sklearn.metrics import (roc_auc_score, average_precision_score, precision_recall_fscore_support, precision_recall_curve)


def evaluate_block(model_name, y_true, y_prob, thresh=0.5):
    y_pred = (y_prob >= thresh).astype(int)
    roc = roc_auc_score(y_true, y_prob)
    ap  = average_precision_score(y_true, y_prob)
    p, r, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="binary", zero_division=0)
    row = {
        "model": model_name,
        "roc_auc": roc,
        "pr_auc": ap,
        "precision": p,
        "recall": r,
        "f1": f1,
        "threshold": thresh
    }
    print(f"[{model_name}] ROC-AUC:{roc:.4f} | PR-AUC:{ap:.4f} | P:{p:.4f} | R:{r:.4f} | F1:{f1:.4f}")
    return row

def save_metrics_table(rows, fname):
    df = pd.DataFrame(rows)
    df.to_csv(fname, index=False)
    print(f" Saved metrics → {fname}")
    return df

def find_best_threshold(y_true, y_prob):
    precision, recall, thresholds = precision_recall_curve(y_true, y_prob)
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
    best_idx = f1_scores.argmax()
    return thresholds[best_idx], f1_scores[best_idx]


XGBoost

In [9]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dval   = xgb.DMatrix(X_val,   label=y_val)
dtest  = xgb.DMatrix(X_test,  label=y_test)

In [10]:

params = {
    "objective": "binary:logistic",
    "eval_metric": ["aucpr", "auc"],
    "tree_method": "hist",         
    "max_depth": 6,            
    "eta": 0.1,                    
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "min_child_weight": 20,       
    "scale_pos_weight": scale_pos_weight,
    "lambda": 1.0,
    "nthread": 4                  
}

bst = xgb.train(
    params,
    dtrain,
    num_boost_round=1000,          # from 2000
    evals=[(dtrain, "train"), (dval, "val")],
    early_stopping_rounds=50,     
    verbose_eval=50
)


[0]	train-aucpr:0.04379	train-auc:0.96785	val-aucpr:0.08700	val-auc:0.97094
[50]	train-aucpr:0.34212	train-auc:0.98145	val-aucpr:0.45113	val-auc:0.98151
[100]	train-aucpr:0.37808	train-auc:0.98410	val-aucpr:0.48666	val-auc:0.98284
[150]	train-aucpr:0.39018	train-auc:0.98590	val-aucpr:0.49861	val-auc:0.98296
[180]	train-aucpr:0.39495	train-auc:0.98667	val-aucpr:0.50199	val-auc:0.98272


Predict and Evaluate

In [ ]:
val_prob  = bst.predict(dval,  iteration_range=(0, bst.best_iteration + 1))
test_prob = bst.predict(dtest, iteration_range=(0, bst.best_iteration + 1))

results = []
results.append(evaluate_block("XGB [VAL @ 0.5]",  y_val,  val_prob,  thresh=0.5))
results.append({
     "strategy": "Class Weighting",
    **evaluate_block("XGB [TEST @ 0.5]", y_test, test_prob, thresh=0.5)})


[XGB [VAL @ 0.5]] ROC-AUC:0.9830 | PR-AUC:0.4945 | P:0.0108 | R:0.9717 | F1:0.0213
[XGB [TEST @ 0.5]] ROC-AUC:0.9878 | PR-AUC:0.6345 | P:0.0131 | R:0.9783 | F1:0.0259


Optimize Threshold

In [ ]:
from sklearn.metrics import precision_score, recall_score

thresholds = np.linspace(0.01, 0.99, 99)
best_thresh = None
best_score = 0

for t in thresholds:
    preds = (val_prob >= t).astype(int)
    precision = precision_score(y_val, preds)
    recall = recall_score(y_val, preds)
    
    # Check if both precision and recall are ≥ 90%
    if precision >= 0.90 and recall >= 0.90:
        f1 = 2 * (precision * recall) / (precision + recall)
        if f1 > best_score:
            best_score = f1
            best_thresh = t

# Report results
if best_thresh is not None:
    print(f" Found threshold with P≥90% and R≥90%")
    print(f"Best threshold: {best_thresh:.4f} | F1: {best_score:.4f}")
    
    # Evaluate using this threshold
    results.append(evaluate_block(f"XGB [VAL @ {best_thresh:.3f} | P≥90 R≥90]", y_val, val_prob, thresh=best_thresh))
    results.append(evaluate_block(f"XGB [TEST @ {best_thresh:.3f} | P≥90 R≥90]", y_test, test_prob, thresh=best_thresh))
else:
    print("⚠️ No threshold found with both precision and recall ≥ 90%.")



Best threshold: 0.9871 | Best F1: 0.5320
[XGB [VAL @ 0.987]] ROC-AUC:0.9830 | PR-AUC:0.4945 | P:0.6801 | R:0.4368 | F1:0.5320
[XGB [TEST @ 0.987]] ROC-AUC:0.9878 | PR-AUC:0.6345 | P:0.7847 | R:0.5350 | F1:0.6362


Resampling-SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE

X_train_df = pd.DataFrame(X_train, columns=feat_names)
y_train_series = pd.Series(y_train)


X_sample = X_train_df.sample(frac=0.6, random_state=42)
y_sample = y_train_series.iloc[X_sample.index]


smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_sample, y_sample)


X_val_df = pd.DataFrame(X_val, columns=feat_names)
X_test_df = pd.DataFrame(X_test, columns=feat_names)

dtrain_resampled = xgb.DMatrix(X_train_res, label=y_train_res)
dval = xgb.DMatrix(X_val_df, label=y_val)
dtest = xgb.DMatrix(X_test_df, label=y_test)


Predict and Evaluate

In [19]:

params_resampling = {
    "objective": "binary:logistic",
    "eval_metric": ["aucpr", "auc"],
    "tree_method": "hist",         
    "max_depth": 6,            
    "eta": 0.1,                    
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "min_child_weight": 20,  
    "lambda": 1.0,
    "nthread": 4                  
}

# Train
bst_smote = xgb.train(
    params_resampling,
    dtrain_resampled,
    num_boost_round=500,
    evals=[(dtrain_resampled, "train"), (dval, "val")],
    early_stopping_rounds=50,
    verbose_eval=50
)


[0]	train-aucpr:0.97188	train-auc:0.97723	val-aucpr:0.08867	val-auc:0.96953
[50]	train-aucpr:0.99553	train-auc:0.99588	val-aucpr:0.39679	val-auc:0.97755
[96]	train-aucpr:0.99884	train-auc:0.99886	val-aucpr:0.40680	val-auc:0.97662


In [ ]:
# Default Threshold (0.5)
results.append(evaluate_block("XGB [VAL @ 0.5 | SMOTE]",  y_val,  val_prob_smote,  thresh=0.5))
results.append({
     "strategy": "Class Weighting",
    **evaluate_block("XGB [TEST @ 0.5 | SMOTE]", y_test, test_prob_smote, thresh=0.5)})


Optimize Threshold

In [ ]:
best_thresh_smote, best_f1_smote = find_best_threshold(y_val, val_prob_smote)
print(f"Best threshold (SMOTE): {best_thresh_smote:.4f} | Best F1: {best_f1_smote:.4f}")


In [ ]:
results.append(evaluate_block(f"XGB [VAL @ {best_thresh_smote:.3f} | SMOTE]", y_val, val_prob_smote, thresh=best_thresh_smote))
results.append(evaluate_block(f"XGB [TEST @ {best_thresh_smote:.3f} | SMOTE]", y_test, test_prob_smote, thresh=best_thresh_smote))


In [ ]:

params = {
    "objective": "binary:logistic",
    "eval_metric": ["aucpr","auc"],
    "tree_method": "hist",        
    "max_depth": 8,
    "eta": 0.08,              
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "min_child_weight": 10,
    "scale_pos_weight": scale_pos_weight,
    "lambda": 1.0

}

bst = xgb.train(
    params,
    dtrain,
    num_boost_round=2000,
    evals=[(dtrain,"train"), (dval,"val")],
    early_stopping_rounds=100,
    verbose_eval=50
)

[0]	train-aucpr:0.08878	train-auc:0.97478	val-aucpr:0.17054	val-auc:0.97475
[50]	train-aucpr:0.35587	train-auc:0.98501	val-aucpr:0.46223	val-auc:0.98192
[100]	train-aucpr:0.40968	train-auc:0.98732	val-aucpr:0.50440	val-auc:0.98261
[150]	train-aucpr:0.42132	train-auc:0.98935	val-aucpr:0.51185	val-auc:0.98271
[200]	train-aucpr:0.42787	train-auc:0.99083	val-aucpr:0.51089	val-auc:0.98203
[250]	train-aucpr:0.43697	train-auc:0.99191	val-aucpr:0.51161	val-auc:0.98128
[254]	train-aucpr:0.43739	train-auc:0.99198	val-aucpr:0.51166	val-auc:0.98122


In [ ]:
# threshold 0.5
val_prob  = bst.predict(dval,  iteration_range=(0, bst.best_iteration+1))
test_prob = bst.predict(dtest, iteration_range=(0, bst.best_iteration+1))

results = []
results.append(evaluate_block("XGB [VAL]",  y_val,  val_prob,  thresh=0.5))
results.append(evaluate_block("XGB [TEST]", y_test, test_prob, thresh=0.5))

joblib.dump(bst, "../data/models/xgb_model.joblib")
print(" XGBoost model was as ../data/models/xgb_model.joblib")

[XGB [VAL]] ROC-AUC:0.9828 | PR-AUC:0.5124 | P:0.0128 | R:0.9474 | F1:0.0253
[XGB [TEST]] ROC-AUC:0.9876 | PR-AUC:0.6466 | P:0.0155 | R:0.9576 | F1:0.0304
 XGBoost model → ../data/models/xgb_full.json


LightGBM

In [ ]:
import lightgbm as lgb

train_lgb = lgb.Dataset(X_train, label=y_train, free_raw_data=False)
val_lgb   = lgb.Dataset(X_val,   label=y_val,   reference=train_lgb, free_raw_data=False)

params_lgb = {
    "objective": "binary",
    "metric": ["auc","average_precision"],  # AP ~ PR-AUC
    "learning_rate": 0.08,
    "num_leaves": 64,
    "max_depth": -1,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 1,
    "min_data_in_leaf": 200,
    "lambda_l2": 1.0,
    "scale_pos_weight": scale_pos_weight,
    "verbosity": -1
}

gbm = lgb.train(
    params_lgb,
    train_lgb,
    num_boost_round=5000,
    valid_sets=[train_lgb, val_lgb],
    valid_names=["train","val"],
    early_stopping_rounds=200,
    verbose_eval=50
)

val_prob_lgb  = gbm.predict(X_val,  num_iteration=gbm.best_iteration)
test_prob_lgb = gbm.predict(X_test, num_iteration=gbm.best_iteration)

results.append(evaluate_block("LGBM [VAL]",  y_val,  val_prob_lgb))
results.append(evaluate_block("LGBM [TEST]", y_test, test_prob_lgb))

gbm.save_model("../models/lgbm_full.txt")
print("💾 Saved LightGBM model → ../models/lgbm_full.txt")


TypeError: train() got an unexpected keyword argument 'early_stopping_rounds'

RandomForest (baseline + quick importance)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=200, max_depth=12, max_features="sqrt",
    n_jobs=-1, random_state=42, class_weight="balanced_subsample"
)
rf.fit(X_train, y_train)

val_prob_rf  = rf.predict_proba(X_val)[:,1]
test_prob_rf = rf.predict_proba(X_test)[:,1]

results.append(evaluate_block("RF [VAL]",  y_val,  val_prob_rf))
results.append(evaluate_block("RF [TEST]", y_test, test_prob_rf))

# Save RF for reference
joblib.dump(rf, "../models/rf_full.joblib")
print("💾 Saved RandomForest → ../models/rf_full.joblib")


Compare models & pick winner

In [ ]:
tbl = save_metrics_table(results, "../reports/model_val_test_metrics.csv")
display(tbl.sort_values(["pr_auc","f1","roc_auc"], ascending=False).head(10))

# Select best by PR-AUC on VAL first, then F1 as tiebreaker
val_rows = [r for r in results if "VAL" in r["model"]]
best_val = sorted(val_rows, key=lambda r: (r["pr_auc"], r["f1"], r["roc_auc"]), reverse=True)[0]
best_name = best_val["model"].split(" ")[0]   # "XGB" or "LGBM" or "RF"
best_name


Feature Importance

In [ ]:
# Map XGB/LGBM feature indices to names
def dump_gain_importance_xgb(bst, feat_names):
    imp = bst.get_score(importance_type="gain")  # dict f{idx} -> score
    rows = []
    for k,v in imp.items():
        idx = int(k[1:]) if k.startswith("f") else None
        name = feat_names[idx] if (idx is not None and idx < len(feat_names)) else k
        rows.append((name, v))
    df_imp = pd.DataFrame(rows, columns=["feature","gain_importance"]).sort_values("gain_importance", ascending=False)
    df_imp.to_csv("../reports/xgb_gain_importance.csv", index=False)
    print("📄 Saved XGB gain importance → ../reports/xgb_gain_importance.csv")
    return df_imp

def dump_gain_importance_lgb(gbm, feat_names):
    imp = gbm.feature_importance(importance_type="gain")
    df_imp = pd.DataFrame({"feature": feat_names[:len(imp)], "gain_importance": imp})
    df_imp = df_imp.sort_values("gain_importance", ascending=False)
    df_imp.to_csv("../reports/lgbm_gain_importance.csv", index=False)
    print("📄 Saved LGBM gain importance → ../reports/lgbm_gain_importance.csv")
    return df_imp

if best_name == "XGB":
    top_imp = dump_gain_importance_xgb(bst, feat_names)
elif best_name == "LGBM":
    top_imp = dump_gain_importance_lgb(gbm, feat_names)
else:
    # RF impurity-based importance
    imp = getattr(rf, "feature_importances_", None)
    if imp is not None:
        df_imp = pd.DataFrame({"feature": feat_names[:len(imp)], "rf_importance": imp}).sort_values("rf_importance", ascending=False)
        df_imp.to_csv("../reports/rf_importance.csv", index=False)
        top_imp = df_imp
        print("📄 Saved RF importance → ../reports/rf_importance.csv")
    else:
        top_imp = None

display(top_imp.head(25) if top_imp is not None else "No importance table.")


Permutation Importance on small VAL subset

In [ ]:

from sklearn.inspection import permutation_importance

VAL_PI_MAX = 100_000
idx = np.random.choice(X_val.shape[0], min(VAL_PI_MAX, X_val.shape[0]), replace=False)

# Choose estimator compatible with sklearn interface
if best_name == "XGB":
    # Quick wrapper to use predict_proba-like behavior
    # We'll use the trained XGB model's prediction in a lambda
    class XGBWrapper:
        def __init__(self, booster): self.model = booster
        def predict_proba(self, X):
            dX = xgb.DMatrix(X)
            p = self.model.predict(dX, iteration_range=(0, self.model.best_iteration+1))
            return np.c_[1-p, p]
    est = XGBWrapper(bst)

elif best_name == "LGBM":
    class LGBWrapper:
        def __init__(self, booster): self.model = booster
        def predict_proba(self, X):
            p = self.model.predict(X, num_iteration=self.model.best_iteration)
            return np.c_[1-p, p]
    est = LGBWrapper(gbm)

else:
    est = rf

pi = permutation_importance(
    est, X_val[idx], y_val[idx],
    n_repeats=1, random_state=42,
    scoring="average_precision", n_jobs=-1
)
pi_df = pd.DataFrame({
    "feature": feat_names[:len(pi.importances_mean)],
    "perm_importance_pr_auc": np.abs(pi.importances_mean)
}).sort_values("perm_importance_pr_auc", ascending=False)

pi_df.to_csv("../reports/permutation_importance_small_val.csv", index=False)
print("📄 Saved permutation importance (small VAL) → ../reports/permutation_importance_small_val.csv")
display(pi_df.head(20))


SMOTE experiment on a tiny train subset

In [ ]:
# Only to demonstrate resampling; do NOT run on full 31M (too heavy).
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression

N_SUB = 300_000  # small subset for demo
# stratified sample
pos_idx = np.where(y_train == 1)[0]
neg_idx = np.where(y_train == 0)[0]

n_pos = len(pos_idx)
n_neg_target = max(n_pos * 100, 1_000)  # cap majority for speed
sel_neg = np.random.choice(neg_idx, size=min(n_neg_target, len(neg_idx)), replace=False)
sub_idx = np.r_[pos_idx, sel_neg]
np.random.shuffle(sub_idx)

X_sub = X_train[sub_idx]
y_sub = y_train[sub_idx]

sm = SMOTE(random_state=42, n_jobs=-1)
X_sm, y_sm = sm.fit_resample(X_sub, y_sub)

# A quick model on resampled data (LogReg saga)
lr_sm = LogisticRegression(
    solver="saga", max_iter=200, tol=1e-3, n_jobs=-1, C=0.5
)
lr_sm.fit(X_sm, y_sm)

val_prob_sm = lr_sm.predict_proba(X_val)[:,1]
_ = evaluate_block("SMOTE(LogReg) [VAL]", y_val, val_prob_sm)


Save final “winner” predictions & thresholds

In [ ]:
# Pick winner by highest PR-AUC on VAL among XGB/LGBM/RF
summary = pd.read_csv("../reports/model_val_test_metrics.csv")
summary.sort_values(["pr_auc","f1","roc_auc"], ascending=False, inplace=True)
display(summary.head(6))

winner = summary.iloc[0]["model"]
if winner.startswith("XGB"):
    final_val_prob  = val_prob
    final_test_prob = test_prob
    chosen = "xgb_full.json"
elif winner.startswith("LGBM"):
    final_val_prob  = val_prob_lgb
    final_test_prob = test_prob_lgb
    chosen = "lgbm_full.txt"
else:
    final_val_prob  = val_prob_rf
    final_test_prob = test_prob_rf
    chosen = "rf_full.joblib"

# Optional: tune threshold to maximize F1 on VAL
from sklearn.metrics import f1_score
thr_grid = np.linspace(0.01, 0.50, 50)
f1s = [f1_score(y_val, (final_val_prob>=t).astype(int)) for t in thr_grid]
best_thr = float(thr_grid[int(np.argmax(f1s))])

final_row_val  = evaluate_block(f"WINNER({winner}) [VAL tuned]",  y_val,  final_val_prob,  best_thr)
final_row_test = evaluate_block(f"WINNER({winner}) [TEST tuned]", y_test, final_test_prob, best_thr)

pd.DataFrame([final_row_val, final_row_test]).to_csv("../reports/winner_tuned_metrics.csv", index=False)
json.dump({"winner": winner, "model_file": chosen, "best_threshold": best_thr},
          open("../models/winner_meta.json","w"))
print(f"🏁 Winner: {winner} | Model file: {chosen} | Best threshold (VAL): {best_thr:.3f}")


In [ ]:
from sklearn.metrics import precision_score, recall_score

thresholds = np.linspace(0.01, 0.99, 99)
high_score = []

for t in thresholds:
    preds = (val_prob_weighted >= t).astype(int)
    precision = precision_score(y_val, preds)
    recall = recall_score(y_val, preds)
    
    if precision >= 0.90 and recall >= 0.90:
        high_score.append((t, precision, recall))

# Show results
if high_score:
    print("Thresholds with both precision and recall ≥ 90%:")
    for t, p, r in high_score:
        print(f"Threshold: {t:.4f} | Precision: {p:.4f} | Recall: {r:.4f}")
else:
    print("No threshold found with both precision and recall ≥ 90%.")
